# Imviz Demonstration
#### STScI Onsite Demo Day 2022-10-27
https://jdaviz.readthedocs.io/en/latest/imviz/index.html

This notebook presents some basic functionality of the Imviz tool of Jdaviz. It is intended to be used within a presentation flow after the basic layout and subset functionality has been demonstrated. Please review Ricky O'Steen's presentation to get a background on the Imviz layout and how to use Subsets.

For additional resources, please review our provided full example notebooks maintained in the Jdaviz repository: https://github.com/spacetelescope/jdaviz/tree/main/notebooks

Import modules needed for this notebook:

In [ ]:
import tempfile
import warnings

from astroquery.mast import Observations

from jdaviz import Imviz

We create an Imviz instance and grab the default viewer instance as well:

In [ ]:
imviz = Imviz()

Let's download some data and load it into Imviz:

In [ ]:
files = ['jw02727-o002_t062_nircam_clear-f090w_i2d.fits',
         'jw02727-o002_t062_nircam_clear-f277w_i2d.fits'
        ]

for fn in files:
    # Uncomment these lines if you didn't download the data from Box
    #uri = f"mast:JWST/product/{fn}"
    #result = Observations.download_file(uri, local_path=fn)
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(fn)

Now, we can display Imviz:

In [ ]:
imviz.show()

## WCS Linking Controls
https://jdaviz.readthedocs.io/en/latest/imviz/plugins.html#link-control

The first thing you will probably notice is that the image doesn't take up the entire viewer area. If you press the "b" key to blink to the other loaded image, you will see that this image is zoomed correctly by default. The odd default zoom on the other is because the images are linked by pixel when loaded. We can instead link by WCS (world coordinates) so that the images will be properly aligned.

There are two ways to link in Imviz. Firstly, you can do it from within the UI through the Links Control plugin by selecting "WCS". Either open the plugin tray to the right and search for "Links Control", or execute the following cell to open it automatically:

In [ ]:
imviz.plugins['Links Control'].open_in_tray()

You can also set the link type from the API using the Imviz Helper:

In [ ]:
imviz.link_data(link_type='wcs')

Now if you blink, your two images should be aligned

## Aperture Photometry
https://jdaviz.readthedocs.io/en/latest/imviz/plugins.html#simple-aperture-photometry

With the ability to define subsets, as demonstrated in Ricky's overview, you can perform simple aperture photometry interactively on your image. To show the plugin, either navigate to it on the plugin bar, or execute the following cell to open it automatically:

In [ ]:
imviz.plugins['Imviz Simple Aperture Photometry'].open_in_tray()

#### Defining Photometry Target
1. Firstly, ensure, the correct dataset you would like to perform photometry on is selected in the plugin.
2. Next, define a subset on the image using the subset tool. Center the subset on the object to perform photometry on
3. This subset should now be selectable as the Aperture under the plugin. Select that subset

#### Defining Background
Once the Aperture is selected, you should now be presented with the ability to define the background information for your photometry calculation. There are currently three different ways to define the background:
* By default, the manual options are presented for full control over the background values
* Annulus provides a width around the subset to average to automatically calculate a background value
* If there is a particular part of the sky you would like to use to define the background on, you can draw a separate subset and select that as the background subset

#### Scaling and Count Conversion
To calibrate the final values, we provide the opportunity to modify the Flux Scaling and Count Conversion factors used in the photometry calculation. Adjust these values if desired

#### Generate Plot type
Imviz provides different types of plots to render: Curve of growth, Radial Profile (using a binned average), and Raw Radial Profile (Raw will show all points used to calculate the Radial Profile curve with no binning.

At this point, you can also fit a guassian curve to the curve, if desired. Click "Calculate" to generate your plot

#### Photometry Results
Imviz will pass your inputs to the photutils backend to perform the photometry. The results of the photometry run will be shown below the plot.

## Plot Options
The Plot Options plugins allows you to modify how your image appears. Here, we'll demo something a little prettier, but first, we need to download some new datasets

In [ ]:
files = ['jw02731-o001_t017_nircam_clear-f090w_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f187n_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f200w_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f335m_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f444w_i2d.fits',
         'jw02731-o001_t017_nircam_f444w-f470n_i2d.fits'
        ]
# Uncomment these lines if you didn't download the data from Box
#for f in files:
#    result = Observations.download_file(f'mast:JWST/product/{f}', base_url="https://mast.stsci.edu/api/v0.1/Download/file")
    
imviz_carina = Imviz(verbosity='error')
for f in files:
    imviz_carina.load_data(f, data_label=f[31:36])


# Link images by WCS (without affine approximation)
imviz_carina.plugins['Links Control'].link_type = 'WCS'
imviz_carina.plugins['Links Control'].wcs_use_affine = False

imviz_carina.show()

To show the Plot Options plugin, either search in the plugin bar, or execute the following cell to open it automatically:

In [ ]:
plot_options = imviz_carina.plugins['Plot Options']
plot_options.open_in_tray()

Here you can adjust the color, opacity, contrast, and bias among other things. You can adjust them manually via the UI, or programmatically via the API to make image appear as you'd like:

In [ ]:
plot_options.image_color_mode = 'Monochromatic'
img_settings = {'f090w': {'image_color': '#cc7ace',
                          'stretch_vmax': 4,
                          'image_opacity': 0.32,
                          'image_contrast': 0.69,
                          'image_bias': 0.39},
                'f187n': {'image_color': '#66cfef',
                          'stretch_vmin': 0,
                          'stretch_vmax': 16,
                          'image_opacity': 0.4,
                          'image_contrast': 0.94,
                          'image_bias': 0.74},
                'f200w': {'image_color': '#61d3e1',
                          'stretch_vmax': 6,
                          'image_opacity': 0.31,
                          'image_contrast': 2.15,
                          'image_bias': 0.66},
                'f335m': {'image_color': '#afff6b',
                          'stretch_vmin': 4,
                          'stretch_vmax': 27,
                          'image_opacity': 0.23,
                          'image_contrast': 3.13,
                          'image_bias': 0.67},
                'f444w': {'image_color': '#ff767c',
                          'stretch_vmax': 10,
                          'image_opacity': 0.18,
                          'image_contrast': 1.77,
                          'image_bias': 0.56},
                'f470n': {'image_color': '#f7787d',
                          'stretch_vmin': 1,
                          'stretch_vmax': 11,
                          'image_opacity': 0.4,
                          'image_contrast': 2.53,
                          'image_bias': 0.27}}

for layer, settings in img_settings.items():
    plot_options.layer = f'{layer}[DATA]'
    for k,v in settings.items():
        setattr(plot_options, k, v)